In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./tf_mnist", one_hot=True)

Extracting ./tf_mnist\train-images-idx3-ubyte.gz
Extracting ./tf_mnist\train-labels-idx1-ubyte.gz
Extracting ./tf_mnist\t10k-images-idx3-ubyte.gz
Extracting ./tf_mnist\t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 1e-2
training_epochs = 25
batch_size = 100
display_epoch = 1
logs_path = './logs'

In [4]:
graph = tf.Graph()
with graph.as_default():
    
    Xtr = tf.placeholder(tf.float32, [None, 784], name='n_Xtr')
    Ytr = tf.placeholder(tf.float32, [None, 10],  name='n_tr')
    
    t_w = tf.Variable(tf.random_normal([784,10], name='Weights'))
    t_b = tf.Variable(tf.zeros([10], name='Biases'))
    
    with tf.name_scope('Model'):
        pred = (tf.matmul(Xtr, t_w)+t_b)
    
    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Ytr))
        
    with tf.name_scope('SGD'):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
        
    with tf.name_scope('Accuracy'):
        correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(Ytr,1))
        acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
    init = tf.global_variables_initializer()
    
    # Create a summary to monitor cost tensor
    tf.summary.scalar("loss", cost)
    tf.summary.scalar("accuracy", acc)
    
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()

In [5]:
with tf.Session(graph=graph) as sess:
    sess.run(init)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    total_batch = int(mnist.train.num_examples/batch_size)
    for epoch in range(training_epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, cost_, summary_, acc_ = sess.run([optimizer, cost, merged_summary_op, acc],
                            feed_dict={ Xtr: batch_x, Ytr: batch_y })
            
            # Write logs at every iteration
            summary_writer.add_summary(summary_, epoch*total_batch+i)
            avg_cost += cost_/total_batch
        if (epoch+1) % display_epoch == 0:
            print ("Epoch: %4d, cost: %.4f, acc: %.4f"% (epoch+1, avg_cost, acc_))
    
    print ("Optimization Finished!")
    
    # Test model
    # Calculate accuracy
    test_acc = acc.eval(feed_dict={Xtr: mnist.test.images, Ytr: mnist.test.labels})
    print ("Accuracy: %.4f"% (test_acc))
            
    print ("Run the command line:\n",
           "--> tensorboard --logdir=%s"% (logs_path))

Epoch:    1, cost: 9.5191, acc: 0.2100
Epoch:    2, cost: 4.7122, acc: 0.4100
Epoch:    3, cost: 3.2151, acc: 0.6400
Epoch:    4, cost: 2.5123, acc: 0.5700
Epoch:    5, cost: 2.1142, acc: 0.5200
Epoch:    6, cost: 1.8591, acc: 0.6600
Epoch:    7, cost: 1.6806, acc: 0.6900
Epoch:    8, cost: 1.5480, acc: 0.6600
Epoch:    9, cost: 1.4449, acc: 0.7100
Epoch:   10, cost: 1.3624, acc: 0.7500
Epoch:   11, cost: 1.2946, acc: 0.7700
Epoch:   12, cost: 1.2375, acc: 0.7000
Epoch:   13, cost: 1.1887, acc: 0.7700
Epoch:   14, cost: 1.1464, acc: 0.8200
Epoch:   15, cost: 1.1091, acc: 0.7400
Epoch:   16, cost: 1.0762, acc: 0.8300
Epoch:   17, cost: 1.0466, acc: 0.8200
Epoch:   18, cost: 1.0199, acc: 0.7700
Epoch:   19, cost: 0.9957, acc: 0.7700
Epoch:   20, cost: 0.9735, acc: 0.7800
Epoch:   21, cost: 0.9533, acc: 0.8400
Epoch:   22, cost: 0.9344, acc: 0.8400
Epoch:   23, cost: 0.9171, acc: 0.8300
Epoch:   24, cost: 0.9009, acc: 0.8800
Epoch:   25, cost: 0.8858, acc: 0.8300
Optimization Finished!
Ac